In [15]:
# Import libraries
import pgmpy.models
import pgmpy.inference
import numpy as np
from enum import Enum

In [27]:
numSlices = 24

class NodeNames(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    MC = "MC"
    CC = "CC"

In [30]:
# Create a dynamic bayesian network
model = pgmpy.models.DynamicBayesianNetwork()
# Add nodes
model.add_nodes_from([NodeNames.MITM.value, NodeNames.SRM.value, NodeNames.UC.value, NodeNames.UPS.value])
# Add edges
for i in range(0, numSlices):
    if i != (numSlices - 1):
        for nodeName in NodeNames:
            model.add_edges_from([((nodeName.value, i), (nodeName.value, i+1))])
    
    model.add_edges_from([((NodeNames.MITM.value, i), (NodeNames.SRM.value, i)), ((NodeNames.MITM.value, i), (NodeNames.UC.value, i)),
                          ((NodeNames.SRM.value, i), (NodeNames.UPS.value, i)), ((NodeNames.UC.value, i), (NodeNames.UPS.value, i))])

# Print edges
print('--- Edges ---')
print(model.edges())
print()
            

--- Edges ---
[(<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(MITM, 1) at 0x7f188b18beb0>), (<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(SRM, 0) at 0x7f188b126b50>), (<DynamicNode(MITM, 0) at 0x7f188b18b040>, <DynamicNode(UC, 0) at 0x7f188b126490>), (<DynamicNode(SRM, 0) at 0x7f188b18bee0>, <DynamicNode(SRM, 1) at 0x7f188b126970>), (<DynamicNode(SRM, 0) at 0x7f188b18bee0>, <DynamicNode(UPS, 0) at 0x7f188b126910>), (<DynamicNode(UC, 0) at 0x7f188b18b190>, <DynamicNode(UC, 1) at 0x7f188b1260d0>), (<DynamicNode(UC, 0) at 0x7f188b18b190>, <DynamicNode(UPS, 0) at 0x7f188b15a640>), (<DynamicNode(UPS, 0) at 0x7f188b18b130>, <DynamicNode(UPS, 1) at 0x7f188b126bb0>), (<DynamicNode(MITM, 1) at 0x7f188b18beb0>, <DynamicNode(SRM, 1) at 0x7f188b1267c0>), (<DynamicNode(MITM, 1) at 0x7f188b18beb0>, <DynamicNode(UC, 1) at 0x7f188b126e50>), (<DynamicNode(SRM, 1) at 0x7f188b126970>, <DynamicNode(UPS, 1) at 0x7f188b126ca0>), (<DynamicNode(UC, 1) at 0x7f188b1260d0>, <DynamicNode(UPS, 1

PySmile Network

In [1]:
import pysmile
import pysmile_license
import os

In [21]:
DEBUG = False

In [16]:
def print_node_info(net, node_handle):
    print("Node id/name: " + net.get_node_id(node_handle) + "/" +
    net.get_node_name(node_handle))
    print(" Outcomes: " + " ".join(net.get_outcome_ids(node_handle)))
    parent_ids = net.get_parent_ids(node_handle)
    if len(parent_ids) > 0:
        print(" Parents: " + " ".join(parent_ids))
    child_ids = net.get_child_ids(node_handle)
    if len(child_ids) > 0:
        print(" Children: " + " ".join(child_ids))
    print_cpt_matrix(net, node_handle)
    
def print_cpt_matrix(net, node_handle):
    cpt = net.get_node_definition(node_handle)
    parents = net.get_parents(node_handle)
    dim_count = 1 + len(parents)
    dim_sizes = [0] * dim_count
    for i in range(0, dim_count - 1):
        dim_sizes[i] = net.get_outcome_count(parents[i])
    dim_sizes[len(dim_sizes) - 1] = net.get_outcome_count(node_handle)
    coords = [0] * dim_count
    for elem_idx in range(0, len(cpt)):
        index_to_coords(elem_idx, dim_sizes, coords)
        outcome = net.get_outcome_id(node_handle, coords[dim_count - 1])
        out_str = " P(" + outcome
        if dim_count > 1:
            out_str += " | "
            for parent_idx in range(0, len(parents)):
                if parent_idx > 0:
                    out_str += ","
                parent_handle = parents[parent_idx]
                out_str += net.get_node_id(parent_handle) + "=" + \
                net.get_outcome_id(parent_handle, coords[parent_idx])
        prob = cpt[elem_idx]
        out_str += ")=" + str(prob)
        print(out_str)
        
def index_to_coords(index, dim_sizes, coords):
    prod = 1
    for i in range(len(dim_sizes) - 1, -1, -1):
        coords[i] = int(index / prod) % dim_sizes[i]
        prod *= dim_sizes[i]

In [23]:
# Create and read the DBN from file
net = pysmile.Network()
net.read_file(os.getcwd() + "/../../../Genie-DBN/DBN-MITM.xdsl")

if DEBUG:
    for n in net.get_all_nodes():
        print_node_info(net, n)
    

25